In [ ]:
import torch
from torch.utils.data import IterableDataset, DataLoader, Subset
from datetime import datetime as dt, timedelta
import pandas as pd
import os
import random
import numpy as np
import torch.nn as nn
from pandas import DataFrame as df
import mplfinance as mpf

# check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

seed = 42  # choose any seed you prefer
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

##### Dataset

In [ ]:
class PriceDataset(torch.utils.data.Dataset):
    def __init__(self, item, timespan, start_date_str, end_date_str):
        self.directory = f'../csvfiles/{item}'
        self.item = item
        self.timespan = timespan
        start_date = dt.strptime(start_date_str, '%Y-%m-%d').date()
        end_date = dt.strptime(end_date_str, '%Y-%m-%d').date()
        self.dates = [single_date.strftime("%Y-%m-%d") for single_date in self.daterange(start_date, end_date)]
        self.columns = [1, 4]  # Selecting open and close prices
        self.filenames = self.get_filenames()

    def daterange(self, start_date, end_date):
        for n in range(int((end_date - start_date).days) + 1):
            yield start_date + timedelta(n)

    def get_filenames(self):
        filenames = []
        for date in self.dates:
            filename = f"{self.directory}/{self.item}-{self.timespan}-{date}.csv"
            if os.path.exists(filename):
                filenames.append(filename)
        return filenames

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        filename = self.filenames[idx]
        df = pd.read_csv(filename, usecols=self.columns, header=None)
        df = df[df.columns[::-1]]  # Swap the columns
        df = df.diff(axis=1)[1]  # Compute difference between close and open price for each row
        return torch.tensor(df.values, dtype=torch.float32)  # Convert to tensor


def sliding_window_fn(batch):
    windows = []
    for tensor in batch:
        for i in range(tensor.shape[0] - 100 + 1):  # Create windows of 100 rows each
            windows.append(tensor[i:i+100])
    return torch.stack(windows)



#### Assign Dataset and Dataloader

In [ ]:

# Create the dataset
dataset = PriceDataset('BTCUSDT', '1m', '2021-03-01', '2023-04-30')

# Shuffle the dataset indices
indices = list(range(len(dataset)))
random.shuffle(indices)

# Split the indices into training and test sets
split_idx = int(0.8 * len(indices))
train_indices, test_indices = indices[:split_idx], indices[split_idx:]

# Create data subsets using the indices
train_data = Subset(dataset, train_indices)
test_data = Subset(dataset, test_indices)

# Create the data loaders
train_loader = DataLoader(train_data, batch_size=1, collate_fn=sliding_window_fn, shuffle=False, drop_last=True)
test_loader = DataLoader(test_data, batch_size=1, collate_fn=sliding_window_fn, shuffle=False, drop_last=True)

##### Checking Dataset and Dataloader

In [ ]:
print(dataset)
print(dataset.directory)

dataset.__getitem__(2)
print(train_data)
print(train_loader)

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=50, output_dim=10, num_layers=5):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        # Define the output layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Reshape the input tensor to [batch_size, sequence_length, number_of_features]
        x = x.view(x.size(0), -1, 1)

        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)

        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)

        # LSTM layer
        out, _ = self.lstm(x, (h0, c0))

        # Index hidden state of last time step
        out = out[:, -1, :]
        out = self.fc(out)
        return out


In [ ]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    for batch_idx, data in enumerate(train_loader):
        data = data.to(device)  # Move the data to the device (CPU or GPU)
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(data)  # Forward pass
        loss = criterion(outputs, data[:, -10:])  # Compute the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update the weights

def evaluate(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)  # Move the data to the device
            outputs = model(data)  # Forward pass
            loss = criterion(outputs, data[:, -10:])  # Compute the loss
            test_loss += loss.item() * data.size(0)  # Accumulate the loss
    return test_loss / len(test_loader.dataset)  # Return the average loss


In [ ]:
# Create the model, criterion, and optimizer
model = LSTM().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
epochs = 40

# Train and evaluate the model
for epoch in range(epochs):  # Adjust the number of epochs as needed
    train(model, train_loader, criterion, optimizer, device)
    val_loss = evaluate(model, test_loader, criterion, device)
    print(f"Epoch {epoch+1}, Validation Loss: {val_loss}")

#### drop out 추가하기